# コロナ禍におけるBCPの因果効果を推定する

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
import statsmodels.api as sm

## 1. 傾向スコアを算出する

In [5]:
path = 'C:/Users/koeci/Google ドライブ/MBA/ワークショップ/data/Analysis/analysis_data_three.csv'
df = pd.read_csv(path, encoding='cp932')

## 分析するアウトカムと、（BCPの傾向スコアを算出するための）説明変数、群別変数の定義
X_cols = [
    'lag_yj_total_assets',
    'lag_yj_operating_cash_flow', 'lag_yj_ros', 'lag_yj_cash_deposit_ratio',
    'lag_yj_leverage', 'lag_yj_stock_price_growth',
    'lag_yj_net_profit_growth', 'lag_yj_firm_age',
    'lag_yj_fixed_assets_ratio', 'lag_yj_extraordinary_loss',
    'lag_yj_foreign_stock_ratio',
    'turnover', 'lag_turnover', 'lag_turnover_cumsum', 'sensitivity_analysis',
    'lag_sensitivity_analysis', 'lag_sensitivity_analysis_cumsum'
    ]

# ダミー変数の処理
dummies = [c for c in df.columns if 'acc_' in c or 'ind_' in c or 'pre_' in c or 'yer_' in c]
for d_col in dummies:
    df[d_col] = df[d_col].apply(lambda x: np.nan if x == 0 else x)
df.dropna(axis=1, how='all', inplace=True)
df.fillna(value=0, inplace=True)
dummies = [c for c in df.columns if 'acc_' in c or 'ind_' in c or 'yer_' in c]
for d_col in range(2007, 2009):
    d_col = 'yer_' + str(d_col)
    dummies.remove(d_col)
X_cols = X_cols + dummies

# 設定
X = df[X_cols]
Z = df['BCP'].astype('int').astype('category')

# ロジスティック回帰を用いて傾向スコアを算出する
model = LogisticRegression(random_state=1234)
model.fit(X=X, y=Z)
ps = pd.Series(model.predict_proba(X=X)[:, 1])
ps.name = 'ps'

# psをdfにmerge
df = pd.concat([df, ps], axis=1)

# 2020~2021年に売上高成長率がマイナスになった企業に限定する
df = df.query('year == 2021 and b_sales_growth < 0')
q = 1
q_min = q / 100
q_max = 1 - q / 100
df = df[(df['b_sales_growth'] >= df['b_sales_growth'].quantile(q_min)) & (df['b_sales_growth'] <= df['b_sales_growth'].quantile(q_max))]
ps = df['ps']
y = df['b_sales_growth']
Z = df['BCP'].astype('int').astype('category')

C:\Users\koeci\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


## 2. 傾向スコアマッチング

In [7]:
table = pd.concat([ps, Z, y], axis=1)
table.reset_index(drop=True, inplace=True)
interval = np.arange(0, 1.05, 0.05)
match_list = []
for i in range(0, len(interval) - 1):
    temp0 = table[(table['BCP'] == 0) & (interval[i] < table['ps']) & (table['ps'] < interval[i + 1])]
    temp1 = table[(table['BCP'] == 1) & (interval[i] < table['ps']) & (table['ps'] < interval[i + 1])]
    if (len(temp0) > 0) & (len(temp1) > 0):
        match_list.append(temp1['b_sales_growth'].mean() - temp0['b_sales_growth'].mean())

print('ATT = {:.3f} ± {:.3f} (s.d={:.3f})'.format(np.nanmean(match_list), np.nanstd(match_list) * 1.96, np.std(match_list)))
print(len(match_list))

ATT = 0.034 ± 0.125 (s.d=0.064)
14


In [10]:
ATT_list = []
sample_size = len(df[df['BCP'] == 1])

for i in range(5000):
    idx1 = pd.Series(df.loc[df['BCP'] == 1, 'b_sales_growth'].index).sample(n=sample_size, replace=True, random_state=i)
    idx0 = pd.Series(df.loc[df['BCP'] == 0, 'b_sales_growth'].index).sample(n=sample_size, replace=True, random_state=i)
    
    Z_tmp = np.r_[Z[idx1], Z[idx0]]
    y_tmp = np.r_[y[idx1], y[idx0]]
    ps_tmp = np.r_[ps[idx1], ps[idx0]]
    w01_tmp = (1 - Z_tmp) * ps_tmp / (1 - ps_tmp)

    E1 = np.mean(y_tmp[Z_tmp == 1])
    E0 = np.sum(y_tmp * w01_tmp) / np.sum(w01_tmp)
    ATT = E1 - E0
    ATT_list.append(ATT)

print('ATT = {:.4f} ± {:.4f} (s.d.={:.4f})'.format(np.nanmean(ATT_list), np.nanstd(ATT_list)*1.98, np.nanstd(ATT_list)))

ATT = 0.0122 ± 0.0116 (s.d.=0.0059)


In [276]:
# DR推定量の取得を関数化
def get_dr(df, y_column, x_columns, treat_column, dummy_columns=[]):
    """
    DR推定量を算出する関数

    Parameters:
    ----------
    df : dataframe
        目的変数、共変量、介入有無のフラグ、傾向スコアのカラムが入ったデータフレーム
    y_column : str
        目的変数のカラム名
    x_columns : list
        説明変数のカラム名が格納されているリスト
    treat_column : str
        介入有無のフラグがあるカラム名
    dummy_columns : list
        説明変数うち、ダミー変数にする必要のあるカラム名が格納されているリスト

    Returns:
    ----------
    result : int
    """
    ## DR推定量の算出
    n = len(df)
    # 結果変数
    y = df[y_column]
    #共変量
    X = pd.get_dummies(df[x_columns], columns=dummy_columns, drop_first=True)
    # 介入効果
    z = df[treat_column]
    # 傾向スコアの算出
    ps_model = LogisticRegression(random_state=1234).fit(X, z)
    ps_score = ps_model.predict_proba(X)[:, 1]
    # 介入群のみのデータ
    group1_df = df[df[treat_column]==1]
    y1 = group1_df[y_column]
    X1 = pd.get_dummies(group1_df[x_columns], columns=dummy_columns, drop_first=True)
    # 対照群のみのデータ
    group0_df = df[df[treat_column]==0]
    y0 = group0_df[y_column]
    X0 = pd.get_dummies(group0_df[x_columns], columns=dummy_columns, drop_first=True)
    # 介入群に対するモデル
    model1 = sm.OLS(y1, X1).fit()
    # 対照群に対するモデル
    model0 = sm.OLS(y0, X0).fit()
    # データ全体をpredictする
    fitted1 = model1.predict(X)
    fitted0 = model0.predict(X)
    # 推定
    dre1 = sum(z * y / ps_score + (1 - z  / ps_score)* fitted1) / n
    dre0 = sum((1 - z) /(1 - ps_score) * y + (1 - (1 - z)  / (1 - ps_score))* fitted0) / n

    result = dre1 - dre0
    print(result)

y_column = 'b_sales_growth'
x_columns = X_cols
treat_column = 'BCP'
get_dr(df, y_column, x_columns, treat_column)

0.017900860508287764
